In [4]:
import numpy as np
import seaborn as sns
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time

# scraping salaries from home page

In [5]:
url = 'https://www.basketball-reference.com/'
res = requests.get(url)
res.status_code

200

In [11]:
teams = []
def sports_page(url):
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'lxml')
    wc_table = soup.find('table', {'id' : 'confs_standings_W'})
    ec_table = soup.find('table', {'id' : 'confs_standings_E'})
    
    #wc teams
    for row in wc_table.find_all('tr')[1:]:
        wc_team = {}
        wc_team['slug'] = row.find('a').attrs['title']
        wc_team['link'] = row.find('a').attrs['href']
        teams.append(wc_team)
    #ec teams
    for row in ec_table.find_all('tr')[1:]:
        ec_team = {}
        ec_team['slug'] = row.find('a').attrs['title']
        ec_team['link'] = row.find('a').attrs['href']
        teams.append(ec_team)
        
    df = pd.DataFrame(teams)
        
        
    return df
        
team_links = 'https://www.basketball-reference.com/'

In [16]:
df = sports_page(url)

In [19]:
df.head()

,slug,link
0,Los Angeles Lakers,/teams/LAL/2020.html
1,Los Angeles Clippers,/teams/LAC/2020.html
2,Denver Nuggets,/teams/DEN/2020.html
3,Dallas Mavericks,/teams/DAL/2020.html
4,Houston Rockets,/teams/HOU/2020.html


In [18]:
df.to_csv('../data/team_links.csv')

## find salary table on team page

This will be salary based on the 2019-2020 season

In [21]:
url = 'https://www.basketball-reference.com'
driver = webdriver.Chrome(executable_path="../chromedriver/windows/chromedriver.exe")
play = []
for page in df['link']:
    time.sleep(3)
    team_page = url + page
    driver.get(team_page)
    html = driver.page_source
    html = BeautifulSoup(html, 'lxml')
    ad_table = html.find('table', {'id' : 'salaries2'})
    chart = ad_table.find('tbody')
    players = chart.find_all('tr')
    for stat in players:
        number = [row.text for row in stat.find_all('td')]
        play.append(number)

In [22]:
salaries = pd.DataFrame(play)
salaries.columns = ['Player', 'salary']
salaries.to_csv('data/salaries.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'data/salaries.csv'